# 동영상 뽑다가 마지막에 실패

In [2]:
import re
import pandas as pd
from pymongo import MongoClient
from bs4 import BeautifulSoup
import requests


client = MongoClient('localhost', 27017)
db = client.movies

collection = db.info
df = pd.DataFrame(list(collection.find())).drop(columns='_id')

# 전체 df
title_filter = df.english_name.apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
title_list = list(title_filter.apply(lambda x: re.sub(r'\s+', '_', x)))
date = list(df.date.apply(lambda x:[x[:4], str(int(x[:4])-1)]))

# # 외국영화만
# idx_list = []
# country_list = list(df['info'].apply(lambda x: x.split(', ')[2:]))
# for idx, country in enumerate(country_list):
#     if '한국' not in country and len(country) != 0:
#         idx_list.append(idx)
        
# # 외국영화만
# title_filter = df.iloc[idx_list, :].english_name.apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
# title_list = list(title_filter.apply(lambda x: re.sub(r'\s+', '_', x)))
# date = list(df.iloc[idx_list, :].date.apply(lambda x:x[:4]))

list_zip = zip(title_list, date)

items = []

for name, date in list_zip:
    url = 'https://www.rottentomatoes.com/m/' + name
    data = requests.get(url)
    if data.status_code == 200:
        soup = BeautifulSoup(data.text, 'html.parser')
        year = soup.find('p', class_='info').text.split(',')[0]
        
        if year in date:
            popcorn = soup.find('score-board-deprecated')['audiencescore']
            tomato = soup.find('score-board-deprecated')['tomatometerscore']
            
            items.append({
                'tomato' : tomato,
                'popcorn' : popcorn
                })
        else:
            items.append({
            'tomato' : '',
            'popcorn' : ''
            })
    else:
        items.append({
            'tomato' : '',
            'popcorn' : ''
            })
        
df_t = pd.DataFrame(items)
df_concat = pd.concat([df, df_t], axis=1)
df_filter = df_concat[df_concat['popcorn'] !='']
df_filter_dict = df_filter.to_dict(orient='records')

collection = db.boxoffice
collection.insert_many(df_filter_dict)

In [10]:
import re
import pandas as pd
from pymongo import MongoClient
import pandas as pd
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

client = MongoClient('localhost', 27017)
db = client.movies

collection = db.reviews
df = pd.DataFrame(list(collection.find())).drop(columns='_id')

df.review = df.review.apply(lambda x: x[:3])
df_t = df.to_dict(orient='records')

items = []
for dic in df_t:
    for review in dic['review']:
        items.append({
            'name' : dic['name'],
            'review' : review
        })
data = pd.DataFrame(items)
data.review = data.review.apply(lambda x: re.sub(r'\s+', ' ', x))

def remove_last_num(sentence):
    cleaned_sentence = sentence.rstrip(' 0123456789')
    while cleaned_sentence and cleaned_sentence[-1].isdigit():
        cleaned_sentence = cleaned_sentence.rstrip(' 0123456789')
    return cleaned_sentence

data['review'] = data['review'].apply(remove_last_num)


#모델, 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("matthewburke/korean_sentiment")
model = AutoModelForSequenceClassification.from_pretrained("matthewburke/korean_sentiment")

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    
    # 확률
    proabilities = torch.softmax(outputs.logits, dim=1)
    # 라벨링  긍정1 부정0
    sentiment_label = torch.argmax(proabilities, dim=1).item()

    return proabilities.detach().numpy(), sentiment_label

# data[review]열 감정분석 후 data[['per', 'sentiment']] 열 추가
data[['per', 'sentiment']] = data['review'].apply(analyze_sentiment).apply(pd.Series)

# data.to_csv('review_sentiment.csv', index=False)

# 긍부정도 평가
unique_movie_ids = data['movie_id'].unique()

positive = []
negative = []

for movie_id in unique_movie_ids:
    selected_rows = data.loc[data['movie_id'] == movie_id]
    
    # sentiment 컬럼 값에 대한 카운트
    sentiment_counts = selected_rows['sentiment'].value_counts()
    
    # 각 sentiment 값의 비율 계산
    positive_percentage = sentiment_counts.get(1, 0) / len(selected_rows) * 100
    negative_percentage = sentiment_counts.get(0, 0) / len(selected_rows) * 100
    
    positive.append(positive_percentage)
    negative.append(negative_percentage)

for i, movie_id in enumerate(unique_movie_ids):
    print(f"Movie ID {movie_id}: 긍정(1)의 비율: {positive[i]:.2f}%, 부정(0)의 비율: {negative[i]:.2f}%")
